In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, optimizers, callbacks, losses, metrics
import keras_tuner as kt

In [13]:
train_data = pd.read_csv("./datasets/train.csv")
test_data = pd.read_csv("./datasets/test.csv")

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [15]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [33]:
X_train = train_data.drop(columns = "label").values
y_train = train_data["label"].values

X_test = test_data.values

In [34]:
mean, std = np.mean(X_train), np.std(X_train)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [35]:
X_test.shape

(28000, 784)

In [36]:
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

In [38]:
model = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = (5, 5), activation = "relu", name = "block1_conv1"),
    layers.Conv2D(filters = 64, kernel_size = (5, 5), activation = "relu", name = "block1_conv2"),
    layers.MaxPool2D(pool_size = 2, name = "block1_maxpooling"),
    layers.Dropout(rate = 0.2, name = "block1_dropout"),
    layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu", name = "block2_conv1"),
    layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu", name = "block2_conv2"),
    layers.MaxPool2D(pool_size = 2, name = "block2_maxpooling"),
    layers.Dropout(rate = 0.2, name = "block2_dropout"),
    layers.Flatten(name = "Flatten"),
    layers.Dense(units = 256, activation = "relu", name = "fc1"),
    layers.Dropout(rate = 0.2, name = "fc_dropout"),
    layers.Dense(units = 10, activation = "softmax", name = "prediction")
])

model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.001, epsilon = 1e-08, decay = 1e-5),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [
        metrics.SparseCategoricalAccuracy(name = "accuracy")
    ]
)

In [41]:
history = model.fit(
    x = X_train,
    y = y_train,
    validation_split = 0.2,
    epochs = 30,
    batch_size = 64,
    callbacks = [
        callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True),
        callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 3, min_lr = 0.00001)
    ]
)

Epoch 1/30
525/525 [==============================] - 7s 11ms/step - loss: 0.1970 - accuracy: 0.9386 - val_loss: 0.0681 - val_accuracy: 0.9777 - lr: 0.0010
Epoch 2/30
525/525 [==============================] - 5s 10ms/step - loss: 0.0626 - accuracy: 0.9807 - val_loss: 0.0409 - val_accuracy: 0.9887 - lr: 0.0010
Epoch 3/30
525/525 [==============================] - 5s 10ms/step - loss: 0.0465 - accuracy: 0.9853 - val_loss: 0.0338 - val_accuracy: 0.9900 - lr: 0.0010
Epoch 4/30
525/525 [==============================] - 5s 10ms/step - loss: 0.0365 - accuracy: 0.9883 - val_loss: 0.0347 - val_accuracy: 0.9911 - lr: 0.0010
Epoch 5/30
525/525 [==============================] - 5s 10ms/step - loss: 0.0312 - accuracy: 0.9899 - val_loss: 0.0492 - val_accuracy: 0.9850 - lr: 0.0010
Epoch 6/30
525/525 [==============================] - 6s 11ms/step - loss: 0.0280 - accuracy: 0.9914 - val_loss: 0.0282 - val_accuracy: 0.9920 - lr: 0.0010
Epoch 7/30
525/525 [==============================] - 6s 11ms/st

In [48]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis=1)

875/875 [==============================] - 2s 3ms/step


In [49]:
prediction

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [54]:
pred_df = pd.DataFrame([], columns = ["ImageId", "Label"])
pred_df["ImageId"] = test_data.index + 1
pred_df["Label"] = prediction
pred_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [55]:
pred_df.to_csv("./results/standarization_two_conv_blocks.csv", index = False)